# From a Number Formatting Example to Function Composition

*Adapted from https://github.com/bencode/code/blob/main/content/format-number.ipynb*

Suppose we need to implement a number formatting function with the following requirements:

- Format options including percentage and suffixes like `wan`, `yi`, etc.
- Configurable decimal precision
- Rounding methods: round, ceiling, and floor
- Thousands separator
- Custom prefixes and suffixes

See the TypeScript definition for details.

## Number Formatting

### Interface Definition

Please excuse that this example is in a Chinese context

```ts
export type FormatRules = Partial<{
  format: string
  precision: number
  carry: 'floor' | 'ceil' | 'round'
  thousandsSeparator: boolean
}>

type FormatEnum =
  | 'percent'
  | 'wan'     // 10k (Chinese: 万)
  | 'baiwan'  // 1M (Chinese: 百万)
  | 'qianwan' // 10M (Chinese: 千万)
  | 'yi'      // 100M (Chinese: 亿)

export function formatNumber(value: number, opts: FormatRules);
```

### format suffix

Suitable suffixes will be added after formatting

In [1]:
function getFormatSuffix(format) {
  const map = {
    percent: '%',
    wan: '万',
    baiwan: '百万',
    qianwan: '千万',
    yi: '亿',
  }
  return map[format]
}

getFormatSuffix('percent')

'%'

In [2]:
getFormatSuffix('yi') // Sorry for using Chinese pinyin, it's a bit evil.

'亿'

### auto

When format is not specified, we need to "infer" an appropriate format - of course, this only applies to the units mentioned above.

In [3]:
function detectFormat(value) {
  const list = [
    [1_0000_0000, 'yi'],
    [1000_0000, 'qianwan'],
    [100_0000, 'baiwan'],
    [1_0000, 'wan'],
  ]
  const item = list.find(pair => value >= pair[0])
  return item ? item[1] : undefined
}

In [4]:
detectFormat(123456789)

'yi'

In [5]:
detectFormat(98765)

'wan'

In [6]:
detectFormat(123)

### Thousands Separator

A regex-based solution would be more elegant than a loop-based one

In [7]:
function formatWithThousandsSeparator(num/*:string*/) {
  const [integerPart, decimalPart] = num.split('.')
  const formattedInteger = integerPart.replace(/\B(?=(\d{3})+(?!\d))/g, ',')
  return decimalPart ? `${formattedInteger}.${decimalPart}` : formattedInteger
}

In [8]:
formatWithThousandsSeparator('123456789')

'123,456,789'

Note: Credit to GPT for this regex - lookbehind/lookahead syntax isn't my strong suit 😅

### Rounding

While we can use Number's built-in functions for rounding and floor/ceiling operations,
note that these methods only work with integers, whereas our requirement is to round to specific decimal places.

In [9]:
function getCarryValue(value, type) {
  if (type === 'floor') {
    return Math.floor(value)
  }
  if (type === 'ceil') {
    return Math.ceil(value)
  }
  return Math.round(value)
}

### Format Scaling

Now that we have our utility methods ready, let's focus on the core task: scaling data based on format specifications.

In [10]:
function getScaledValue(value, format) {
  const shift = getFormatShift(format)
  const next = shift !== 0 ? value * 10 ** shift : value
  return next
}

function getFormatShift(format) {
  const map = {
    percent: 2,
    // 万
    wan: -4,
    // 百万
    baiwan: -6,
    // 千万
    qianwan: -7,
    // 亿
    yi: -8,
  }
  return format ? map[format] ?? 0 : 0
}

In [11]:
getScaledValue(0.785, 'percent')

78.5

In [12]:
getScaledValue(12345, 'wan')

1.2345

### Handle Decimal Precision

Using the rounding function defined earlier `getCarryValue()`

In [13]:
function getPrecisionValue(value, precision, type) {
  const scale = 10 ** precision
  const next = getCarryValue(value * scale, type)
  return next / scale
}

In [14]:
getPrecisionValue(123.123, 2, 'round')

123.12

In [15]:
getPrecisionValue(123.125, 2, 'round')

123.13

### Floating-Point Precision

In [16]:
0.1 + 0.2

0.30000000000000004

Due to JavaScript's floating-point precision issues, and considering our current use case, we'll use round to avoid these problems.

In [17]:
function round(value) {
  const scale = 10 ** 8
  return Math.round(value * scale) / scale
}

In [18]:
round(0.1 + 0.2)

0.3

### Putting It All Together

With all components prepared, it's time to bring everything together.

In [19]:
function formatNumber(value, opts = {}) {
  const format = opts.format === 'auto' ? detectFormat(value) : opts.format
  const precision = opts.precision ?? -1
  const next = getScaledValue(value, format)
  const result = precision >= 0 ? getPrecisionValue(next, precision, opts.carry) : next
  const str = toString(result, precision, format, opts)
  return str
}

Finally, `toString` takes care of the formatting

```ts
export type FormatRules = Partial<{
  format: string
  precision: number
  carry: 'floor' | 'ceil' | 'round'
  thousandsSeparator: boolean
}>
```

In [20]:
function toString(num, precision, format, opts) {
  const value = precision >= 0 ? num.toFixed(precision) : `${round(num)}` // <-- 注意第二个分支
  const next = opts.thousandsSeparator ? formatWithThousandsSeparator(value) : value
  const formatSuffix = format ? getFormatSuffix(format) : ''
  return `${next}${formatSuffix}`
}

In [21]:
formatNumber(1234512345.6789, { format: 'wan', precision: 2, carry: 'round', thousandsSeparator: true })

'123,451.23万'

### Default Precision

When a precision parameter is provided, toFixed handles floating-point precision issues internally;  
otherwise, we need to handle it ourselves - this is why we use `${round(num)}` above.

In [22]:
0.2 + 0.7

0.8999999999999999

In [23]:
(0.2 + 0.7) * 234

210.59999999999997

In [24]:
formatNumber((0.2 + 0.7) * 234)

'210.6'

## Function Composition

Using function composition to refactor this scenario may feel a bit contrived, but being deliberate helps with practice.

(Continued...)

1